In [1]:
import os
import json
import time
import requests
from datetime import datetime
from datetime import date
import dateutil
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np

In [ ]:
# Choose time period of interest
start_year=1900
start_month=1
end_year=2010
end_month=12

In [ ]:
# functtions to clean headlines
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

import re, string, random

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
# code to tokenize data
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
#pr=parse_response(send_request(['2012', '1']))
#p2=' '.join(pr)
#ptest=tknzr.tokenize(p2)
#len(pr)
import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [ ]:
# functions to gather NYT headlines, clean them, and save them by year in a JSON file

# function to give inputs for NYT database 
def train_years_NYT(mstart, ystart, mstop, ystop):
    start=datetime.date(ystart, mstart, 1)
    end=datetime.date(ystop, mstop, 1)
    months_in_range = [x.split(' ') for x in pd.date_range(start, end, freq='MS').strftime("%Y %#m").tolist()]
    
    return months_in_range

def send_request(date):
    '''Sends a request to the NYT Archive API for given date.'''
    base_url = 'https://api.nytimes.com/svc/archive/v1/'
    url = base_url + '/' + date[0] + '/' + date[1] + '.json?api-key=' + 'API Key here' 
    response = requests.get(url).json()
    time.sleep(6)
    return response


def parse_response(response):
    '''Parses and returns response as pandas data frame.''' 
        #'date': []} 
        #'doc_type': [],
        #'material_type': [],
        #'section': [],
        #'keywords': []}
    
    articles = response['response']['docs'] 
    ls=[]
    for article in articles: # For each article, make sure it falls within our date range
        date = dateutil.parser.parse(article['pub_date']).date()
        #if is_valid(article, date):
        #data['date'].append(date)
        ls.append(article['headline']['main']) 
    full_cloud=' '.join(ls)
    token_cloud=tknzr.tokenize(full_cloud)
    red_cloud=remove_noise(token_cloud, stop_words)
    #red_cloud_2=get_tweets_for_model(red_cloud)
    strung=' '.join(red_cloud)
    #data.append(strung)    
    #df=pd.DataFrame(data) 
    return strung

import csv 
import json 


def get_data(dates):
    '''Sends and parses request/response to/from NYT Archive API for given dates.'''
    total = 0
    print('Date range: ' + str(dates[0]) + ' to ' + str(dates[-1]))
    open('headlineData.json','w')
    #if not os.path.exists('headlines'):
     #   os.mkdir('headlines')
    list=[]    
    for date in dates:
        response = send_request(date)
        df = parse_response(response)
        total += 1
        list.append(df)
        print('Saving headlines from ' + date[0] + '-' + date[1])
    with open('headlineData.json', 'a', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(list, indent=4)
        jsonf.write(jsonString)
        
    print('Months of articles collected: ' + str(total))

In [ ]:
#generate NYT headline file
#get_data(train_years_NYT(start_month,start_year,end_month,end_year))

In [ ]:
# Data features:
with open('headlineData.json') as f:
    data = json.load(f)

In [ ]:
from collections import Counter
# function to determine number of distinct words and their frequency 
def word_data(data, months, n):
    total_dat=data[0]
    i=1
    while i<months:
        total_dat=total_dat+data[i]
        i+=1
    count= len(Counter((total_dat).split()).values())
    most_comm=Counter((total_dat).split()).most_common(n)
        
    return count, most_comm


In [ ]:
count, most_comm= word_data(data, 1332, 1000)
print(count)
print(most_comm)